# Transformer 모델 학습하기

In [1]:
# #구글 드라이버를 마운트
# from google.colab import drive

# drive.mount('gdrive')

In [2]:
pwd

'C:\\Users\\USER\\Desktop\\KOR\\200605_중간점검'

In [3]:
import pandas as pd
lines= pd.read_csv('../all_data.csv')
len(lines)

29005

In [4]:
lines = lines.drop('data',axis=1)
lines = lines.values
lines

array([['12시 땡!', '하루가 또 가네요.'],
       ['1지망 학교 떨어졌어', '위로해 드립니다.'],
       ['3박4일 놀러가고 싶다', '여행은 언제나 좋죠.'],
       ...,
       ['한국에 돌아오고 나서 증상이 악화된 거 같아.', '너무 심하시면 병원을 다시 가보는 건 어떨까요?'],
       ['약을 끊었더니 증상이 예전보다 더 심해진 것 같아.', '너무 심하시면 병원을 다시 가보는 건 어떨까요?'],
       ['통증이 점점 심해지는것 같아.', '너무 심하시면 병원을 다시 가보는 건 어떨까요?']], dtype=object)

In [5]:
# We will be using TensorFlow 2.0 for this tutorial!
!pip install tf-nightly-2.0-preview

ERROR: Could not find a version that satisfies the requirement tf-nightly-2.0-preview (from versions: none)
ERROR: No matching distribution found for tf-nightly-2.0-preview


In [6]:
import tensorflow as tf
print("You have version", tf.__version__)
assert tf.__version__ >= "2.0" # TensorFlow ≥ 2.0 required

You have version 2.2.0


In [7]:
import numpy as np
import unicodedata, re

from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Lambda, Layer, Embedding, LayerNormalization


# 데이터 전처리


### 형태소 분석기 Mecab 사용하기
형태소 분석기로 Mecab을 사용하였습니다.

Mecab은 별도로 다운로드 해주셔야 사용할 수 있습니다.
(https://cleancode-ws.tistory.com/97 를 참고해주세요)

In [9]:
import MeCab
import re

mecab = MeCab.Tagger()

def mecab_morphs(text):
    morphs = []
    
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+") 
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
        
    # 추출한 token중에 문자열만 선택.
    for token in temp:
        morphs.append(token[0])

    return morphs

def mecab_tagging(text):
    morphs = []
    
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+") 
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
        
    # 추출한 token중에 문자열만 선택.
    for token in temp:
        morphs.append((token[0], token[1]))
    
    return morphs

In [10]:
def preprocess(s):
  # for details, see https://www.tensorflow.org/alpha/tutorials/sequences/nmt_with_attention
  # s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
  s = re.sub(r"([?.!¡,¿])", r" \1 ", s) # Add spaces around punctuations
  s = re.sub(r'[" "]+', " ", s) # Remove extra space
  s = re.sub(r"[^a-zA-Z?.!ㄱ-ㅎㅏ-ㅣ가-힣]+", " ", s) # Remove other characters
  s = " ".join(mecab_morphs(s))
  s = s.strip()
  s = '<start> ' + s + ' <end>'
  return s

print("Original:", lines[0])
sentences = [(preprocess(en), preprocess(es)) for (en, es) in lines]
print("Preprocessed:", sentences[0])

Original: ['12시 땡!' '하루가 또 가네요.']
Preprocessed: ('<start> 시 땡 ! <end>', '<start> 하루 가 또 가 네요 . <end>')


In [11]:
source_sentences, target_sentences = list(zip(*sentences))

# In this illustration, I choose not to specify num_words and oov_token due to the size of data.
# for details, please visit https://keras.io/preprocessing/text/
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='') 


# 단어 사전 하나로 만들기.
tokenizer.fit_on_texts(source_sentences + target_sentences)

source_data = tokenizer.texts_to_sequences(source_sentences)
target_data = tokenizer.texts_to_sequences(target_sentences)
print("Sequence:", source_data[0])

# 입력값은 앞에 패딩주기
source_data = tf.keras.preprocessing.sequence.pad_sequences(source_data, padding='pre',maxlen=20)
target_data = tf.keras.preprocessing.sequence.pad_sequences(target_data, padding='post',maxlen=20)
print("Padded:", source_data[0])


Sequence: [1, 55, 6302, 89, 2]
Padded: [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    1   55 6302   89    2]


In [12]:
# Machine translation models take the entire source sentence and an incomplete sentence in
# target language as inputs at once, and predict the next word for the incomplete sentence.
# We create labels for the decoder by shifting the target sequence one to the right.
target_labels = np.zeros(target_data.shape)
target_labels[:,0:target_data.shape[1] -1] = target_data[:,1:]

print("Target sequence", target_data[0])
print("Target label", target_labels[0])

vocab_len = len(tokenizer.word_index) + 1
# source_vocab_len = len(source_tokenizer.word_index) + 1
# target_vocab_len = len(target_tokenizer.word_index) + 1

print("Size of vocabulary: ", vocab_len)

Target sequence [  1 205   8 173   8  64   3   2   0   0   0   0   0   0   0   0   0   0
   0   0]
Target label [205.   8. 173.   8.  64.   3.   2.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.]
Size of vocabulary:  9113


In [13]:
## word2vec 
import gensim
ko_model = gensim.models.Word2Vec.load('./word2vec_model_200')
ko_model.wv.vectors.shape #30만 단어 200차원으로 학습한 모델링 가지고 오기

(359672, 200)

In [14]:
embedding_matrix = np.zeros((vocab_len, 200))
# 단어 집합 크기의 행과 200개의 열을 가지는 행렬 생성. 값은 전부 0으로 채워진다.
np.shape(embedding_matrix)

(9113, 200)

In [15]:
def get_vector(word):
    if word in ko_model:
        return ko_model[word]
    else:
        return None

In [16]:
for idx in tokenizer.index_word:
    val = tokenizer.index_word[idx]
    temp = get_vector(val) # 단어(key) 해당되는 임베딩 벡터의 200개의 값(value)를 임시 변수에 저장
    if temp is not None: # 만약 None이 아니라면 임베딩 벡터의 값을 리턴받은 것이므로
        embedding_matrix[idx] = temp # 해당 단어 위치의 행에 벡터의 값을 저장한다.

C:\Users\USER\.conda\envs\team2_real\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
C:\Users\USER\.conda\envs\team2_real\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
# For Gradient Tape training
dataset = tf.data.Dataset.from_tensor_slices((source_data, target_data, target_labels)).batch(32)
# For Keras model.fit()
dataset_2 = tf.data.Dataset.from_tensor_slices((source_data, target_data, target_labels))

# Transformer 모델 생성

In [18]:
# 파라미터 설정
# 컴퓨터 사정상 작은 파라미터를 사용하였습니다

# d_model = 64 # 512 in the original paper
d_model = 200 # 워드 임베딩 200으로 함.
d_k = 16 # 64 in the original paper
d_v = 16 # 64 in the original paper
n_heads = 4 # 8 in the original paper
n_encoder_layers = 2 # 6 in the original paper
n_decoder_layers = 2 # 6 in the original paper

max_token_length = 20 # 512 in the original paper

In [19]:
class SingleHeadAttention(Layer):
  def __init__(self, input_shape=(3, -1, d_model), dropout=.0, masked=None):
    super(SingleHeadAttention, self).__init__()
    self.q = Dense(d_k, input_shape=(-1, d_model), kernel_initializer='glorot_uniform', 
                   bias_initializer='glorot_uniform')
    self.normalize_q = Lambda(lambda x: x / np.sqrt(d_k))
    self.k = Dense(d_k, input_shape=(-1, d_model), kernel_initializer='glorot_uniform', 
                   bias_initializer='glorot_uniform')
    self.v = Dense(d_v, input_shape=(-1, d_model), kernel_initializer='glorot_uniform', 
                   bias_initializer='glorot_uniform')
    self.dropout = dropout
    self.masked = masked
  
  # Inputs: [query, key, value]
  def call(self, inputs, training=None):
    assert len(inputs) == 3
    # We use a lambda layer to divide vector q by sqrt(d_k) according to the equation
    q = self.normalize_q(self.q(inputs[0]))
    k = self.k(inputs[1])
    # The dimensionality of q is (batch_size, query_length, d_k) and that of k is (batch_size, key_length, d_k)
    # So we will do a matrix multication by batch after transposing last 2 dimensions of k
    # tf.shape(attn_weights) = (batch_size, query_length, key_length)
    attn_weights = tf.matmul(q, tf.transpose(k, perm=[0,2,1]))
    if self.masked: # Prevent future attentions in decoding self-attention
      # Create a matrix where the strict upper triangle (not including main diagonal) is filled with -inf and 0 elsewhere
      length = tf.shape(attn_weights)[-1]
      #attn_mask = np.triu(tf.fill((length, length), -np.inf), k=1) # We need to use tensorflow functions instead of numpy
      attn_mask = tf.fill((length, length), -np.inf)
      attn_mask = tf.linalg.band_part(attn_mask, 0, -1) # Get upper triangle
      attn_mask = tf.linalg.set_diag(attn_mask, tf.zeros((length))) # Set diagonal to zeros to avoid operations with infinity
      # This matrix is added to the attention weights so all future attention will have -inf logits (0 after softmax)
      attn_weights += attn_mask
    # Softmax along the last dimension
    attn_weights = tf.nn.softmax(attn_weights, axis=-1)
    if training: # Attention dropout included in the original paper. This is possibly to encourage multihead diversity.
      attn_weights = tf.nn.dropout(attn_weights, rate=self.dropout)
    v = self.v(inputs[2])
    return tf.matmul(attn_weights, v)

In [20]:
# 멀티 헤드 어텐션

class MultiHeadAttention(Layer):
  def __init__(self, dropout=.0, masked=None):
    super(MultiHeadAttention, self).__init__()
    self.attn_heads = list()
    for i in range(n_heads): 
      self.attn_heads.append(SingleHeadAttention(dropout=dropout, masked=masked))
    self.linear = Dense(d_model, input_shape=(-1, n_heads * d_v), kernel_initializer='glorot_uniform', 
                   bias_initializer='glorot_uniform')
    
  def call(self, x, training=None):
    attentions = [self.attn_heads[i](x, training=training) for i in range(n_heads)]
    concatenated_attentions = tf.concat(attentions, axis=-1)
    return self.linear(concatenated_attentions)

In [21]:
# 인코더 

class TransformerEncoder(Layer):
  def __init__(self, dropout=.1, attention_dropout=.0, **kwargs):
    super(TransformerEncoder, self).__init__(**kwargs)
    self.dropout_rate = dropout
    self.attention_dropout_rate = attention_dropout
  def build(self, input_shape):
    self.multihead_attention = MultiHeadAttention(dropout=self.attention_dropout_rate)
    self.dropout1 = tf.keras.layers.Dropout(self.dropout_rate)
    self.layer_normalization1 = LayerNormalization(input_shape=input_shape, epsilon=1e-6)
    
    self.linear1 = Dense(input_shape[-1] * 4, input_shape=input_shape, activation='relu',
                        kernel_initializer='glorot_uniform', bias_initializer='glorot_uniform')
    self.linear2 = Dense(input_shape[-1], input_shape=self.linear1.compute_output_shape(input_shape),
                        kernel_initializer='glorot_uniform', bias_initializer='glorot_uniform')
    self.dropout2 = tf.keras.layers.Dropout(self.dropout_rate)
    self.layer_normalization2 = LayerNormalization(input_shape=input_shape, epsilon=1e-6)
    super(TransformerEncoder, self).build(input_shape)
  def call(self, x, training=None):
    sublayer1 = self.multihead_attention((x, x, x), training=training)
    sublayer1 = self.dropout1(sublayer1, training=training)
    layernorm1 = self.layer_normalization1(x + sublayer1)
    
    sublayer2 = self.linear2(self.linear1(layernorm1))
    sublayer1 = self.dropout2(sublayer2, training=training)
    layernorm2 = self.layer_normalization2(layernorm1 + sublayer2)
    return layernorm2
  def compute_output_shape(self, input_shape):
    return input_shape

In [22]:
# 디코더

class TransformerDecoder(Layer):
  def __init__(self, dropout=.0, attention_dropout=.0, **kwargs):
    super(TransformerDecoder, self).__init__(**kwargs)
    self.dropout_rate = dropout
    self.attention_dropout_rate = attention_dropout
  def build(self, input_shape):
    self.multihead_self_attention = MultiHeadAttention(dropout=self.attention_dropout_rate, masked=True)
    self.dropout1 = tf.keras.layers.Dropout(self.dropout_rate)
    self.layer_normalization1 = LayerNormalization(input_shape=input_shape, epsilon=1e-6)
    
    self.multihead_encoder_attention = MultiHeadAttention(dropout=self.attention_dropout_rate)
    self.dropout2 = tf.keras.layers.Dropout(self.dropout_rate)
    self.layer_normalization2 = LayerNormalization(input_shape=input_shape, epsilon=1e-6)
    
    self.linear1 = Dense(input_shape[-1] * 4, input_shape=input_shape, activation='relu',
                        kernel_initializer='glorot_uniform', bias_initializer='glorot_uniform')
    self.linear2 = Dense(input_shape[-1], input_shape=self.linear1.compute_output_shape(input_shape),
                        kernel_initializer='glorot_uniform', bias_initializer='glorot_uniform')
    self.dropout3 = tf.keras.layers.Dropout(self.dropout_rate)
    self.layer_normalization3 = LayerNormalization(input_shape=input_shape, epsilon=1e-6)
    super(TransformerDecoder, self).build(input_shape)
  def call(self, x, hidden, training=None):
    sublayer1 = self.multihead_self_attention((x, x, x))
    sublayer1 = self.dropout1(sublayer1, training=training)
    layernorm1 = self.layer_normalization1(x + sublayer1)
    
    sublayer2 = self.multihead_encoder_attention((x, hidden, hidden))
    sublayer2 = self.dropout2(sublayer2, training=training)
    layernorm2 = self.layer_normalization2(layernorm1 + sublayer2)
    
    sublayer3 = self.linear2(self.linear1(layernorm1))
    sublayer3 = self.dropout3(sublayer3, training=training)
    layernorm3 = self.layer_normalization2(layernorm2 + sublayer3)
    return layernorm3
  def compute_output_shape(self, input_shape):
    return input_shape

In [24]:
with tf.device('/gpu:0'):

    # Testing if the dimension matches!
    x = tf.ones((3, 26, d_model))
    x1 = tf.ones((3, 18, d_model))
    single_att = SingleHeadAttention(masked=None)
    multi_att = MultiHeadAttention()
    encoder = TransformerEncoder()
    decoder = TransformerDecoder()
    y = single_att((x, x, x)) # Self attention
    y1 = multi_att((x1, x, x)) # Encoder-decoder attention
    print(tf.shape(y))
    print(tf.shape(y1))
    y2 = encoder(x)
    y3 = decoder(x, y2)

    print(tf.shape(y2))
    print(tf.shape(y3))
    #print(layer.trainable_weights)

tf.Tensor([ 3 26 16], shape=(3,), dtype=int32)
tf.Tensor([  3  18 200], shape=(3,), dtype=int32)
tf.Tensor([  3  26 200], shape=(3,), dtype=int32)
tf.Tensor([  3  26 200], shape=(3,), dtype=int32)


In [25]:
class SinusoidalPositionalEncoding(Layer): #Inspired from https://github.com/graykode/nlp-tutorial/blob/master/5-1.Transformer/Transformer_Torch.ipynb
  def __init__(self):
    super(SinusoidalPositionalEncoding, self).__init__()
    self.sinusoidal_encoding = np.array([self.get_positional_angle(pos) for pos in range(max_token_length)], dtype=np.float32)
    self.sinusoidal_encoding[:, 0::2] = np.sin(self.sinusoidal_encoding[:, 0::2])
    self.sinusoidal_encoding[:, 1::2] = np.cos(self.sinusoidal_encoding[:, 1::2])
    self.sinusoidal_encoding = tf.cast(self.sinusoidal_encoding, dtype=tf.float32) # Casting the array to Tensor for slicing
  def call(self, x):
    return x + self.sinusoidal_encoding[:tf.shape(x)[1]]
    #return x + tf.slice(self.sinusoidal_encoding, [0, 0], [tf.shape(x)[1], d_model])
  def compute_output_shape(self, input_shape):
    return input_shape
  def get_angle(self, pos, dim):
    return pos / np.power(10000, 2 * (dim // 2) / d_model)
  def get_positional_angle(self, pos):
    return [self.get_angle(pos, dim) for dim in range(d_model)]

## Assembling the Full Architecture


In [26]:
class Transformer(Model):
  def __init__(self, dropout=.1, attention_dropout=.0, **kwargs):
    super(Transformer, self).__init__(**kwargs)
    self.encoding_embedding = Embedding(vocab_len, d_model, weights=[embedding_matrix], name='en-embedding')
    self.decoding_embedding = Embedding(vocab_len, d_model, weights=[embedding_matrix], name='de-embedding')
    self.pos_encoding = SinusoidalPositionalEncoding()
    self.encoder = [TransformerEncoder(dropout=dropout, attention_dropout=attention_dropout) for i in range(n_encoder_layers)]
    self.decoder = [TransformerDecoder(dropout=dropout, attention_dropout=attention_dropout) for i in range(n_decoder_layers)]
    self.decoder_final = Dense(vocab_len, input_shape=(None, d_model))
  def call(self, inputs, training=None): # Source_sentence and decoder_input
    source_sentence, decoder_input = inputs
    embedded_source = self.encoding_embedding(source_sentence)
    encoder_output = self.pos_encoding(embedded_source)
    for encoder_unit in self.encoder:
      encoder_output = encoder_unit(encoder_output, training=training)
    
    embedded_target = self.decoding_embedding(decoder_input)
    decoder_output = self.pos_encoding(embedded_target)
    for decoder_unit in self.decoder:
      decoder_output = decoder_unit(decoder_output, encoder_output, training=training)
    if training:
      decoder_output = self.decoder_final(decoder_output)
      decoder_output = tf.nn.softmax(decoder_output, axis=-1)
    else:
      decoder_output = self.decoder_final(decoder_output[:, -1:, :])
      decoder_output = tf.nn.softmax(decoder_output, axis=-1)
    return decoder_output

## 모델 학습

In [32]:
transformer_2 = Transformer() # Instantiating a new transformer model

src_seqs, tgt_seqs, tgt_labels = zip(*dataset_2)
train = [tf.cast(src_seqs, dtype=tf.float32), tf.cast(tgt_seqs, dtype=tf.float32)] # Cast the tuples to tensors

transformer_2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

transformer_2.fit(train, tf.cast(tgt_labels, dtype=tf.float32), verbose=1, batch_size=32, epochs=50)

Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
907/907 [==============================] - 53s 59ms/step - loss: 1.7624 - accuracy: 0.6938
Epoch 2/50
907/907 [==============================] - 53s 59ms/step - loss: 0.9231 - accuracy: 0.8177
Epoch 3/50
907/907 [==============================] - 53s 58ms/step - loss: 0.7452 - accuracy: 0.8468
Epoch 4/50
907/907 [==============================] - 52s 58ms/step - loss: 0.6615

In [36]:
def Transformer_Func(dropout,attention_dropout):
    encoding_embedding = Embedding(vocab_len, d_model)
    decoding_embedding = Embedding(vocab_len, d_model)
    pos_encoding = SinusoidalPositionalEncoding()
    encoder = [TransformerEncoder(dropout=dropout, attention_dropout=attention_dropout) for i in range(n_encoder_layers)]
    decoder = [TransformerDecoder(dropout=dropout, attention_dropout=attention_dropout) for i in range(n_decoder_layers)]
    decoder_final = Dense(vocab_len, input_shape=(None, d_model))
    
    training = False
    for batch, (source_sentence, decoder_input, target_labels) in enumerate(dataset):
#     source_sentence, decoder_input = inputs
        embedded_source = encoding_embedding(source_sentence)
        encoder_output = pos_encoding(embedded_source)
    
        for encoder_unit in encoder:
          encoder_output = encoder_unit(encoder_output, training=training)
    
        embedded_target = decoding_embedding(decoder_input)
        decoder_output = pos_encoding(embedded_target)
        for decoder_unit in decoder:
          decoder_output = decoder_unit(decoder_output, encoder_output, training=training)
        if training:
          decoder_output = decoder_final(decoder_output)
          decoder_output = tf.nn.softmax(decoder_output, axis=-1)
        else:
          decoder_output = decoder_final(decoder_output[:, -1:, :])
          decoder_output = tf.nn.softmax(decoder_output, axis=-1)
        return decoder_output

In [ ]:
transformer.load_weights('transformer_weight.h5')

In [ ]:
transformer_3 = Transformer_Func(dropout=.1, attention_dropout=.0)
# transformer_3.load_weights('transformer_weight.h5')

In [ ]:
%cd gdrive/'My Drive'

## 모델 예측 확인

In [84]:
def translate(model, source_sentence, target_sentence_start=[['<start>']]):
  if np.ndim(source_sentence) == 1: # Create a batch of 1 the input is a sentence
    source_sentence = [source_sentence]
  if np.ndim(target_sentence_start) == 1:
    target_sentence_start = [target_sentence_start]
  # Tokenizing and padding
  source_seq = tokenizer.texts_to_sequences(source_sentence)
  source_seq = tf.keras.preprocessing.sequence.pad_sequences(source_seq, padding='pre', maxlen=15)
  predict_seq = tokenizer.texts_to_sequences(target_sentence_start)
  
  predict_sentence = list(target_sentence_start[0]) # Deep copy here to prevent updates on target_sentence_start
  while predict_sentence[-1] != '<end>' and len(predict_seq) < max_token_length:
    predict_output = model([np.array(source_seq), np.array(predict_seq)], training=None)
    predict_label = tf.argmax(predict_output, axis=-1) # Pick the label with highest softmax score
    predict_seq = tf.concat([predict_seq, predict_label], axis=-1) # Updating the prediction sequence
    predict_sentence.append(tokenizer.index_word[predict_label[0][0].numpy()])
  
  return predict_sentence[1:-1]

In [44]:
for i in range(5):
  print("Source sentence: ", source_sentences[i*10])
  print("Target sentence: ", target_sentences[i*10])
  print("Predicted sentence: ", ' '.join(translate(transformer_2, source_sentences[i*10].split(' '))))
  print()

Source sentence:  <start> 시 땡 ! <end>
Target sentence:  <start> 하루 가 또 가 네요 . <end>
Predicted sentence:  <start> 달달 하루 노래 . <end>

Source sentence:  <start> SNS 보 면 나 만 빼 고 다 행복 해 보여 <end>
Target sentence:  <start> 자랑 하 는 자리 니까요 . <end>
Predicted sentence:  <start> 자랑 하 는 자리 니까요 . <end>

Source sentence:  <start> 가스 비 비싼데 감기 걸리 겠 어 <end>
Target sentence:  <start> 따뜻 하 게 사세요 ! <end>
Predicted sentence:  <start> 이럴 때 잘 쉬 는 게 중요 해요 . <end>

Source sentence:  <start> 가족 들 이랑 서먹 해 <end>
Target sentence:  <start> 다 들 바빠서 이야기 할 시간 이 부족 했 나 봐요 . <end>
Predicted sentence:  <start> 완벽 한 사람 은 아무 도 없 어요 . <end>

Source sentence:  <start> 간만에 떨리 니까 좋 더라 <end>
Target sentence:  <start> 떨리 는 감정 은 그 자체 로 소중 해요 . <end>
Predicted sentence:  <start> 애석 하 지만 당신 을 더 소중히 생각 해 주 세요 . <end>



In [ ]:
for i in range(10):
  print("Source sentence: ", source_sentences[20500+i*5])
  print("Target sentence: ", target_sentences[20500+i*5])
  print("Predicted sentence: ", ' '.join(translate(transformer_3, source_sentences[20500+i*5].split(' '))))
  print()

In [39]:
  print("Predicted sentence: ", ' '.join(translate(transformer_2, '<start> 밥 뭐 먹지? <end>'.split(' '))))

Predicted sentence:  <start> 맛있 게 드세요 . <end>


# 모델 저장

In [ ]:
from keras.models import load_model
import pickle

# 딕셔너리 저장
# tokenizer.word_index, tokenizer.index_word >> 케라스에서 생성해준 딕셔너리를 볼 수 있음.
file = open("word_to_index","wb") 
pickle.dump(tokenizer.word_index,file) 
file.close()
file = open("index_to_word","wb") 
pickle.dump(tokenizer.index_word,file) 
file.close()

# 모델 weight저장
transformer.save_weights("transformer_weight.h5")



# 테스트셋 예측하기

In [65]:
import pandas as pd
tests= pd.read_csv('test.csv',names=['Q'])
tests.head()

,Q
0,미유야 안녕
1,반가워
2,넌 누구니?
3,고마워
4,고맙다


In [66]:
test_X= tests['Q'].values
test_X

array(['미유야 안녕', '반가워', '넌 누구니?', '고마워', '고맙다', '심심해', '점심 뭐 먹지?',
       '오늘 김팀장님한테 혼나서 짜증났어', '사람들 모두 날 싫어 하는거 같아', '오늘 기분이 별로 안좋아',
       '요즘 생각이 많아지는 것 같아', '아.. 운동가야하는데 너무 귀찮아..', '그냥 여행이나 가고 싶다',
       '좋아하는 사람이 날 안좋아하는 거 같아', '여자친구랑 싸웠어..', '과장님때문에 힘들다',
       '부모님을 이해할 수 없어', '취업하기 너무 어렵다', '나를 원하는 사람이 없어', '미세먼지 너무 많아',
       '오늘 몸이 안좋네', '인생이 재미 없다', '오늘 컨디션이 안좋아', '내일은 나아질까?', '내일은 좋아질까?',
       '요새 스트레스가 너무 심해', '재밌는거 뭐 없나?', '부모님이 보고싶어', '오늘 친구랑 싸웠어',
       '친구들이랑 사이가 안좋아', '아빠가 날 힘들게해', '오늘 친구랑 싸웠어', '오늘 할 일이 너무 많다',
       '여자친구랑 헤어졌어', '아...회사가기 싫다', '오늘 학교가기 싫어', '아 내일도 회사 가야해',
       '일 그만 둘까?', '오늘 뭐 하지?', '남자친구 때문에 짜증나', '우울하다', '오늘 하루 힘들었다',
       '집에 혼자있으니까 적적하다', '엄마는 날 이해하지 못하는 것 같아', '아침부터 피곤하다', '오늘 뭐 했어?',
       '넌 잘하는게 뭐야?', '나 힘들어', '진짜 싫다', '김부장 재수없어', '짜증난다'], dtype=object)

In [71]:
# index로 변경 후 pad줌.
x = [(preprocess(x)) for x in test_X]
x = tokenizer.texts_to_sequences(x)
x = tf.keras.preprocessing.sequence.pad_sequences(x, padding='pre',maxlen=max_token_length)

In [72]:
x

array([[   0,    0,    0, ...,   86,  852,    2],
       [   0,    0,    0, ...,    1, 6472,    2],
       [   0,    0,    0, ...,  178,   12,    2],
       ...,
       [   0,    0,    0, ...,  160,   40,    2],
       [   0,    0,    0, ...,    1,   15,    2],
       [   0,    0,    0, ...,  310, 3509,    2]])

In [85]:
# test셋 예측값을 (단어 임베딩된 상태의) 리스트로 출력
def predict_text(x_set):
    y_pred = []
    # 전처리
    x_set = [(preprocess(x)) for x in x_set]
    # 예측하기
    for x in x_set:
        pred_seq = ' '.join(translate(transformer_2, x.split(' '))) # 덱스트형태 리스트
        y_pred.append(pred_seq)
    return y_pred

In [87]:
pred = predict_text(test_X)

In [92]:
tests['A']=pred

In [94]:
tests

,Q,A
0,미유야 안녕,안녕 하 세요 .
1,반가워,반갑 안녕 하 세요 .
2,넌 누구니?,저 는 위 로 봇 입니다 .
3,고마워,감사 해요 .
4,고맙다,감사 해요 .
5,심심해,노래 불러 드릴까요 ? 북치 기박 치 기 헥헥
6,점심 뭐 먹지?,오늘 은 짜장면 어떠세요 ?
7,오늘 김팀장님한테 혼나서 짜증났어,마음 이 헛헛 한가 봐요 .
8,사람들 모두 날 싫어 하는거 같아,그걸 깨닫 다니 대단 하 시 군요 .
9,오늘 기분이 별로 안좋아,정도 가 심하 면 병원 을 찾 아 보 는 것 도 좋 을 것 같 아요 .
